In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import trueskill as ts

In [2]:
raw_csv = pd.read_csv('aflstats_2006_2016.csv')

C:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
raw_csv.head()

,Season,Round,Date,Local start time,Venue,Attendance,Home team,HQ1G,HQ1B,HQ2G,...,Marks Inside 50,One Percenters,Bounces,Goal Assists,Time on Ground %,Substitute,Umpire 1,Umpire 2,Umpire 3,Umpire 4
0,2006,1,2006-04-03,1910,Docklands,35434,Collingwood,5,1,7,...,0,5,0,0,100,0,Martin Ellis,Darren Goldspink,Scott Jeffery,NaN
1,2006,1,2006-04-03,1910,Docklands,35434,Collingwood,5,1,7,...,1,1,0,1,61,0,Martin Ellis,Darren Goldspink,Scott Jeffery,NaN
2,2006,1,2006-04-03,1910,Docklands,35434,Collingwood,5,1,7,...,0,2,1,2,78,0,Martin Ellis,Darren Goldspink,Scott Jeffery,NaN
3,2006,1,2006-04-03,1910,Docklands,35434,Collingwood,5,1,7,...,0,4,1,2,97,0,Martin Ellis,Darren Goldspink,Scott Jeffery,NaN
4,2006,1,2006-04-03,1910,Docklands,35434,Collingwood,5,1,7,...,0,0,0,1,64,0,Martin Ellis,Darren Goldspink,Scott Jeffery,NaN


In [4]:
# How big of a dataset are we talking here?
print len(raw_csv.index), 'rows'
raw_headers = list(raw_csv.columns.values)
print len(raw_headers), 'columns'
print 'Column names are:'
print raw_headers

# So just under 100k player records. 65 or so attributes for each player, but plenty of duplication.

94468 rows
65 columns
Column names are:
['Season', 'Round', 'Date', 'Local start time', 'Venue', 'Attendance', 'Home team', 'HQ1G', ' HQ1B', ' HQ2G', ' HQ2B', ' HQ3G', ' HQ3B', 'HQ4G', 'HQ4B', 'Home score', 'Home coach', 'Home coach DOB', 'Away team', 'AQ1G', ' AQ1B', ' AQ2G', ' AQ2B', ' AQ3G', 'AQ3B', 'AQ4G', 'AQ4B', 'Away score', 'Away coach', 'Away coach DOB', 'First name', 'Surname', 'ID', 'DOB', 'Height', 'Weight', 'Jumper No.', 'Playing for', 'Kicks', 'Marks', 'Handballs', 'Goals', 'Behinds', 'Hit Outs', 'Tackles', 'Rebounds', 'Inside 50s', 'Clearances', 'Clangers', 'Frees For', 'Frees Against', 'Brownlow Votes', 'Contested Possessions', 'Uncontested Possessions', 'Contested Marks', 'Marks Inside 50', 'One Percenters', 'Bounces', 'Goal Assists', 'Time on Ground %', 'Substitute', 'Umpire 1', 'Umpire 2', 'Umpire 3', 'Umpire 4']


In [5]:
# What do we need for TrueSkill? Well, we need:
# - Game info (season, round, date, winner, loser)
# - Player info (First Name, Surname - possibly in one string or maybe just use player ID)
#  - Play info (time on ground is pretty useful for TrueSkill)

In [6]:
# Make a new DataFrame with just the info we need
data_cut = raw_csv.drop(['Local start time', 'Venue', 'Attendance', 'HQ1G', ' HQ1B', ' HQ2G', ' HQ2B', ' HQ3G', ' HQ3B', 'HQ4G', 'HQ4B',
                        'Home coach DOB','AQ1G', ' AQ1B', ' AQ2G', ' AQ2B', ' AQ3G', 'AQ3B', 'AQ4G', 'AQ4B','Away coach DOB', 
                         'Height', 'Weight', 'Jumper No.','Kicks', 'Marks', 'Handballs', 'Goals', 'Behinds', 'Hit Outs', 'Tackles', 
                         'Rebounds', 'Inside 50s', 'Clearances', 'Clangers', 'Frees For', 'Frees Against', 'Brownlow Votes', 
                         'Contested Possessions', 'Uncontested Possessions', 'Contested Marks', 'Marks Inside 50', 'One Percenters', 
                         'Bounces', 'Goal Assists', 'Substitute', 'Umpire 1', 'Umpire 2', 'Umpire 3', 'Umpire 4'],inplace = False,axis=1)
data_cut.head()

,Season,Round,Date,Home team,Home score,Home coach,Away team,Away score,Away coach,First name,Surname,ID,DOB,Playing for,Time on Ground %
0,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Nathan,Bassett,56,1976-12-07,Adelaide,100
1,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Rhett,Biglands,83,1977-09-04,Adelaide,61
2,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Matthew,Bode,90,1979-06-29,Adelaide,78
3,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Brett,Burton,68,1978-05-04,Adelaide,97
4,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Matthew,Clarke,78,1973-09-18,Adelaide,64


In [7]:
# Make a column with the player names from the first and last names of the players
data_cut['Player Name'] = data_cut['First name']+' ' + data_cut['Surname']
data_cut['PlayerUniqueID'] = data_cut['DOB']+data_cut['Player Name']
data_cut.head()

,Season,Round,Date,Home team,Home score,Home coach,Away team,Away score,Away coach,First name,Surname,ID,DOB,Playing for,Time on Ground %,Player Name,PlayerUniqueID
0,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Nathan,Bassett,56,1976-12-07,Adelaide,100,Nathan Bassett,1976-12-07Nathan Bassett
1,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Rhett,Biglands,83,1977-09-04,Adelaide,61,Rhett Biglands,1977-09-04Rhett Biglands
2,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Matthew,Bode,90,1979-06-29,Adelaide,78,Matthew Bode,1979-06-29Matthew Bode
3,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Brett,Burton,68,1978-05-04,Adelaide,97,Brett Burton,1978-05-04Brett Burton
4,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Matthew,Clarke,78,1973-09-18,Adelaide,64,Matthew Clarke,1973-09-18Matthew Clarke


In [8]:
# Write to csv so we don't have to process and cut down the data again
data_cut.to_csv('cut down data.csv')

In [9]:
players = data_cut['PlayerUniqueID']

In [10]:
# How many players?
len(players.unique())

1539

In [11]:
# If we take the lower case of all the players in the league, how many unique values are there?
players_lower = players.str.lower()
print len(players_lower.unique()), 'players in lower(players)'
# This doesn't differ from the non-lower case value, so it would seem that there aren't any 
# typing/input issues in the Player Name column
print len(data_cut['ID'].unique()), 'unique player IDs'
# Somehow there are more unique player IDs than unique players
# Put this down to a change in the player ID system?
# This means we can't use player ID as the unique identifier - must use player name
# Later on, found that there player name collisions
# Decided to make PlayerUniqueID from the player's DOB and Name

1539 players in lower(players)
1539 unique player IDs


In [12]:
# Create skill values for all players in the league
playerskill = {name:ts.Rating() for name in players.unique()}
playerskill

{'1990-12-27Jarrad Redden': trueskill.Rating(mu=25.000, sigma=8.333),
 '1987-04-18Stephen Wrigley': trueskill.Rating(mu=25.000, sigma=8.333),
 '1992-02-25Jack Frost': trueskill.Rating(mu=25.000, sigma=8.333),
 '1986-03-20Beau Maister': trueskill.Rating(mu=25.000, sigma=8.333),
 '1994-03-22Tim Broomhead': trueskill.Rating(mu=25.000, sigma=8.333),
 '1997-02-05Blake Hardwick': trueskill.Rating(mu=25.000, sigma=8.333),
 '1988-01-28Grant Birchall': trueskill.Rating(mu=25.000, sigma=8.333),
 '1990-12-09Jack Redden': trueskill.Rating(mu=25.000, sigma=8.333),
 '1985-10-02Bernie Vince': trueskill.Rating(mu=25.000, sigma=8.333),
 '1986-06-20John Meesen': trueskill.Rating(mu=25.000, sigma=8.333),
 '1983-06-07Luke Vogels': trueskill.Rating(mu=25.000, sigma=8.333),
 '1984-05-25Luke Ball': trueskill.Rating(mu=25.000, sigma=8.333),
 '1990-12-19Chris Yarran': trueskill.Rating(mu=25.000, sigma=8.333),
 '1987-01-05Lukas Markovic': trueskill.Rating(mu=25.000, sigma=8.333),
 '1991-03-12Steven Motlop': tru

In [13]:
data_cut.sort_values(['Date', 'Home team','Playing for'])
data_cut.head()

,Season,Round,Date,Home team,Home score,Home coach,Away team,Away score,Away coach,First name,Surname,ID,DOB,Playing for,Time on Ground %,Player Name,PlayerUniqueID
0,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Nathan,Bassett,56,1976-12-07,Adelaide,100,Nathan Bassett,1976-12-07Nathan Bassett
1,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Rhett,Biglands,83,1977-09-04,Adelaide,61,Rhett Biglands,1977-09-04Rhett Biglands
2,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Matthew,Bode,90,1979-06-29,Adelaide,78,Matthew Bode,1979-06-29Matthew Bode
3,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Brett,Burton,68,1978-05-04,Adelaide,97,Brett Burton,1978-05-04Brett Burton
4,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Matthew,Clarke,78,1973-09-18,Adelaide,64,Matthew Clarke,1973-09-18Matthew Clarke


In [14]:
# If player1 and player2 beat player3 and player4 then:
# trueskill.rate([(player1,player2),(player3,player4)],weights=[(player1%,player2%),(player3%,player4%)])
# Assuming player1 spent player1% of the game playing, etc.

data_cut['Played at home'] = (data_cut['Playing for']==data_cut['Home team'])
data_cut['Home win'] = (data_cut['Home score']>data_cut['Away score'])
data_cut['Is draw'] = (data_cut['Home score']==data_cut['Away score'])
data_cut.head()

,Season,Round,Date,Home team,Home score,Home coach,Away team,Away score,Away coach,First name,Surname,ID,DOB,Playing for,Time on Ground %,Player Name,PlayerUniqueID,Played at home,Home win,Is draw
0,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Nathan,Bassett,56,1976-12-07,Adelaide,100,Nathan Bassett,1976-12-07Nathan Bassett,False,False,False
1,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Rhett,Biglands,83,1977-09-04,Adelaide,61,Rhett Biglands,1977-09-04Rhett Biglands,False,False,False
2,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Matthew,Bode,90,1979-06-29,Adelaide,78,Matthew Bode,1979-06-29Matthew Bode,False,False,False
3,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Brett,Burton,68,1978-05-04,Adelaide,97,Brett Burton,1978-05-04Brett Burton,False,False,False
4,2006,1,2006-04-03,Collingwood,77,Mick Malthouse,Adelaide,111,Neil Craig,Matthew,Clarke,78,1973-09-18,Adelaide,64,Matthew Clarke,1973-09-18Matthew Clarke,False,False,False


In [15]:
# If the player played at home and the home team won, the player won
# If the player did not play at home and the home team did not win, the player won
data_cut['Player win'] = data_cut['Played at home'] == data_cut['Home win']
#data_cut.drop(['Played at home','Home win'], axis = 1, inplace = True)
#Decided I do need 'Played at home' and 'Home win' for later, commented this out

In [16]:
data_cut.drop(['First name','Surname','ID','Home coach','Away coach','Player Name'],axis = 1, inplace = True)
data_cut.head()

,Season,Round,Date,Home team,Home score,Away team,Away score,DOB,Playing for,Time on Ground %,PlayerUniqueID,Played at home,Home win,Is draw,Player win
0,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1976-12-07,Adelaide,100,1976-12-07Nathan Bassett,False,False,False,True
1,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1977-09-04,Adelaide,61,1977-09-04Rhett Biglands,False,False,False,True
2,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1979-06-29,Adelaide,78,1979-06-29Matthew Bode,False,False,False,True
3,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1978-05-04,Adelaide,97,1978-05-04Brett Burton,False,False,False,True
4,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1973-09-18,Adelaide,64,1973-09-18Matthew Clarke,False,False,False,True


In [17]:
data_cut['Game Code'] = data_cut['Date']+data_cut['Home team']+data_cut['Away team']

In [18]:
len(data_cut['Game Code'].unique())
# Data for 2148 games?
# 8 games per round, 22 rounds, 10 years gives 1760 games
# Finals must be included?

2148

In [19]:
data_cut['Round'].unique()
# Confirms finals are included, need to find out what the 1L:23L are? Not sure

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', 'QF', 'PF',
       'EF', 'SF', '23', '24', 'GF', 1L, 2L, 3L, 4L, 5L, 6L, 7L, 8L, 9L,
       10L, 11L, 12L, 14L, 15L, 16L, 17L, 18L, 19L, 20L, 21L, 22L, 23L, 13L], dtype=object)

In [20]:
data_cut['Home win'] = (data_cut['Home score']>data_cut['Away score'])
data_cut.head()

,Season,Round,Date,Home team,Home score,Away team,Away score,DOB,Playing for,Time on Ground %,PlayerUniqueID,Played at home,Home win,Is draw,Player win,Game Code
0,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1976-12-07,Adelaide,100,1976-12-07Nathan Bassett,False,False,False,True,2006-04-03CollingwoodAdelaide
1,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1977-09-04,Adelaide,61,1977-09-04Rhett Biglands,False,False,False,True,2006-04-03CollingwoodAdelaide
2,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1979-06-29,Adelaide,78,1979-06-29Matthew Bode,False,False,False,True,2006-04-03CollingwoodAdelaide
3,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1978-05-04,Adelaide,97,1978-05-04Brett Burton,False,False,False,True,2006-04-03CollingwoodAdelaide
4,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1973-09-18,Adelaide,64,1973-09-18Matthew Clarke,False,False,False,True,2006-04-03CollingwoodAdelaide


In [21]:
def whowon(winteam_input):
    HomeTeam = winteam_input[0]
    AwayTeam = winteam_input[1]
    HomeWin = winteam_input[2]
    if HomeWin:
        Winteam = HomeTeam
    else:
        Winteam = AwayTeam
    return Winteam

def wholost(lossteam_input):
    HomeTeam = lossteam_input[0]
    AwayTeam = lossteam_input[1]
    HomeWin = lossteam_input[2]
    if HomeWin:
        Lossteam = AwayTeam
    else:
        Lossteam = HomeTeam
    return Lossteam

        
data_cut['WinTeam']=data_cut[['Home team','Away team', 'Home win']].apply(whowon,axis=1)
data_cut['LossTeam']=data_cut[['Home team','Away team', 'Home win']].apply(wholost,axis=1)
data_cut.head()

,Season,Round,Date,Home team,Home score,Away team,Away score,DOB,Playing for,Time on Ground %,PlayerUniqueID,Played at home,Home win,Is draw,Player win,Game Code,WinTeam,LossTeam
0,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1976-12-07,Adelaide,100,1976-12-07Nathan Bassett,False,False,False,True,2006-04-03CollingwoodAdelaide,Adelaide,Collingwood
1,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1977-09-04,Adelaide,61,1977-09-04Rhett Biglands,False,False,False,True,2006-04-03CollingwoodAdelaide,Adelaide,Collingwood
2,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1979-06-29,Adelaide,78,1979-06-29Matthew Bode,False,False,False,True,2006-04-03CollingwoodAdelaide,Adelaide,Collingwood
3,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1978-05-04,Adelaide,97,1978-05-04Brett Burton,False,False,False,True,2006-04-03CollingwoodAdelaide,Adelaide,Collingwood
4,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1973-09-18,Adelaide,64,1973-09-18Matthew Clarke,False,False,False,True,2006-04-03CollingwoodAdelaide,Adelaide,Collingwood


In [22]:
data_cut.to_csv('data cleaned stage 1.csv')

In [23]:
data_cut = pd.read_csv('data cleaned stage 1.csv')
data_cut.sort_values(['Game Code', 'Home team','Playing for'])
data_cut.head()

,Unnamed: 0,Season,Round,Date,Home team,Home score,Away team,Away score,DOB,Playing for,Time on Ground %,PlayerUniqueID,Played at home,Home win,Is draw,Player win,Game Code,WinTeam,LossTeam
0,0,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1976-12-07,Adelaide,100,1976-12-07Nathan Bassett,False,False,False,True,2006-04-03CollingwoodAdelaide,Adelaide,Collingwood
1,1,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1977-09-04,Adelaide,61,1977-09-04Rhett Biglands,False,False,False,True,2006-04-03CollingwoodAdelaide,Adelaide,Collingwood
2,2,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1979-06-29,Adelaide,78,1979-06-29Matthew Bode,False,False,False,True,2006-04-03CollingwoodAdelaide,Adelaide,Collingwood
3,3,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1978-05-04,Adelaide,97,1978-05-04Brett Burton,False,False,False,True,2006-04-03CollingwoodAdelaide,Adelaide,Collingwood
4,4,2006,1,2006-04-03,Collingwood,77,Adelaide,111,1973-09-18,Adelaide,64,1973-09-18Matthew Clarke,False,False,False,True,2006-04-03CollingwoodAdelaide,Adelaide,Collingwood


In [24]:
data_wrangled = pd.DataFrame(columns=['Game Code','Winteam Players', 'Lossteam Players','GameDraw','Winteam Players %time','Lossteam Players %time'])
data_wrangled.head()

,Game Code,Winteam Players,Lossteam Players,GameDraw,Winteam Players %time,Lossteam Players %time


In [25]:
data_wrangled['Game Code'] = data_cut['Game Code'].unique()

In [26]:
data_wrangled.head()

,Game Code,Winteam Players,Lossteam Players,GameDraw,Winteam Players %time,Lossteam Players %time
0,2006-04-03CollingwoodAdelaide,NaN,NaN,NaN,NaN,NaN
1,2006-04-09AdelaideWest Coast,NaN,NaN,NaN,NaN,NaN
2,2006-04-16MelbourneAdelaide,NaN,NaN,NaN,NaN,NaN
3,2006-04-22FremantleAdelaide,NaN,NaN,NaN,NaN,NaN
4,2006-04-29AdelaideWestern Bulldogs,NaN,NaN,NaN,NaN,NaN


In [27]:
#gamecode = data_wrangled.loc[1,'Game Code']
def packdata(gamecode):
    # Get all players who played in the game that matches gamecode
    played = data_cut.loc[data_cut['Game Code']==gamecode]
    
    # Split all players into two DataFrames: those in the home team and those in the away team
    home = played.loc[played['Played at home']==True]
    away = played.loc[played['Played at home']==False]
    
    # Check if it was a home win - returns a Boolean
    homewin = played['Home win'].unique()[0]
    
    # Check if it was a draw - returns a Boolean
    isdraw = played['Is draw'].unique()[0]
    
    # Create two new DataFrames based on whether or not the home team won
    # If it was a draw, it will assume that the away team won BUT the IsDraw flag also exists
    if homewin:
        won = home
        lost = away
    else:
        won = away
        lost = home

    # Create four new Series, splitting up the relevant columns from the won and lost dataframes
    wonplayers = won['PlayerUniqueID']
    lostplayers = lost['PlayerUniqueID']
    wonplaytime = won['Time on Ground %']
    lostplaytime = lost['Time on Ground %']
    
    # Pack this mess of Series in to an array and output that
    return [wonplayers,lostplayers,wonplaytime,lostplaytime,isdraw]

In [28]:
data_wrangled['Packed Data'] = data_wrangled['Game Code'].apply(packdata)
data_wrangled.head()

,Game Code,Winteam Players,Lossteam Players,GameDraw,Winteam Players %time,Lossteam Players %time,Packed Data
0,2006-04-03CollingwoodAdelaide,NaN,NaN,NaN,NaN,NaN,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi..."
1,2006-04-09AdelaideWest Coast,NaN,NaN,NaN,NaN,NaN,"[[1978-03-24Michael Braun, 1978-06-30Ben Cousi..."
2,2006-04-16MelbourneAdelaide,NaN,NaN,NaN,NaN,NaN,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi..."
3,2006-04-22FremantleAdelaide,NaN,NaN,NaN,NaN,NaN,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi..."
4,2006-04-29AdelaideWestern Bulldogs,NaN,NaN,NaN,NaN,NaN,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi..."


In [29]:
# Unpack the packed data
for i in range(0,len(data_wrangled.index)):
    # Unpacking the winning players
    data_wrangled['Winteam Players'][i] = data_wrangled.loc[i,['Packed Data'][0]][0]

    # Unpacking the losing players
    data_wrangled['Lossteam Players'][i] = data_wrangled.loc[i,['Packed Data'][0]][1]

    # Unpacking the winteam play %
    data_wrangled['Winteam Players %time'][i] = data_wrangled.loc[i,['Packed Data'][0]][2]

    # Unpacking the lossteam play %
    data_wrangled['Lossteam Players %time'][i] = data_wrangled.loc[i,['Packed Data'][0]][3]

    # Unpacking the "isdraw" flag
    data_wrangled['GameDraw'][i] = data_wrangled.loc[i,['Packed Data'][0]][4]

In [30]:
data_wrangled.head()

,Game Code,Winteam Players,Lossteam Players,GameDraw,Winteam Players %time,Lossteam Players %time,Packed Data
0,2006-04-03CollingwoodAdelaide,0 1976-12-07Nathan Bassett 1 1977-...,11000 1972-07-26Nathan Buckley 11001 ...,False,0 100 1 61 2 78 3 97 4 ...,11000 81 11001 59 11002 71 11003 ...,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi..."
1,2006-04-09AdelaideWest Coast,78166 1978-03-24Michael Braun 78167 ...,22 1976-12-07Nathan Bassett 23 1977-...,False,78166 91 78167 92 78168 79 78169 ...,22 100 23 69 24 96 25 90 26 ...,"[[1978-03-24Michael Braun, 1978-06-30Ben Cousi..."
2,2006-04-16MelbourneAdelaide,44 1976-12-07Nathan Bassett 45 1977-...,44990 1988-02-16Clint Bartram 44991 ...,False,44 100 45 49 46 90 47 90 48 ...,44990 81 44991 90 44992 100 44993 ...,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi..."
3,2006-04-22FremantleAdelaide,66 1976-12-07Nathan Bassett 67 1977-...,27808 1976-03-01Peter Bell 27809 ...,False,66 100 67 66 68 94 69 95 70 ...,27808 100 27809 69 27810 88 27811 ...,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi..."
4,2006-04-29AdelaideWestern Bulldogs,88 1976-12-07Nathan Bassett 89 197...,22242 1982-08-27Matthew Boyd 22243 ...,False,88 100 89 54 90 86 91 52 92...,22242 56 22243 100 22244 75 22245 ...,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi..."


In [31]:
# reindexing all the series that are embedded in the dataframe for referencing purposes
# gotta use df['column'][row] for setting it because df.loc[row,'column'] doesn't work for some reason?
for i in range(0,len(data_wrangled.index)):
    winplayers = data_wrangled.loc[i,'Winteam Players']
    winplayers = winplayers.reset_index()
    winplayers = winplayers['PlayerUniqueID']
    data_wrangled['Winteam Players'][i] = winplayers

    lossplayers = data_wrangled.loc[i,'Lossteam Players']
    lossplayers = lossplayers.reset_index()
    lossplayers = lossplayers['PlayerUniqueID']
    data_wrangled['Lossteam Players'][i] = lossplayers
    
    lossplaypercent = data_wrangled.loc[i,'Lossteam Players %time']
    lossplaypercent = lossplaypercent.reset_index()
    lossplaypercent = lossplaypercent['Time on Ground %']
    data_wrangled['Lossteam Players %time'][i] = lossplaypercent
    
    winplaypercent = data_wrangled.loc[i,'Winteam Players %time']
    winplaypercent = winplaypercent.reset_index()
    winplaypercent = winplaypercent['Time on Ground %']
    data_wrangled['Winteam Players %time'][i] = winplaypercent
    
    
data_wrangled.head()

,Game Code,Winteam Players,Lossteam Players,GameDraw,Winteam Players %time,Lossteam Players %time,Packed Data
0,2006-04-03CollingwoodAdelaide,0 1976-12-07Nathan Bassett 1 1977-...,0 1972-07-26Nathan Buckley 1 ...,False,0 100 1 61 2 78 3 97 4 ...,0 81 1 59 2 71 3 68 4 ...,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi..."
1,2006-04-09AdelaideWest Coast,0 1978-03-24Michael Braun 1 1978...,0 1976-12-07Nathan Bassett 1 1977-...,False,0 91 1 92 2 79 3 98 4 ...,0 100 1 69 2 96 3 90 4 ...,"[[1978-03-24Michael Braun, 1978-06-30Ben Cousi..."
2,2006-04-16MelbourneAdelaide,0 1976-12-07Nathan Bassett 1 1977-...,0 1988-02-16Clint Bartram 1 1...,False,0 100 1 49 2 90 3 90 4 ...,0 81 1 90 2 100 3 100 4 ...,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi..."
3,2006-04-22FremantleAdelaide,0 1976-12-07Nathan Bassett 1 1977-...,0 1976-03-01Peter Bell 1 19...,False,0 100 1 66 2 94 3 95 4 ...,0 100 1 69 2 88 3 70 4 ...,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi..."
4,2006-04-29AdelaideWestern Bulldogs,0 1976-12-07Nathan Bassett 1 1977-...,0 1982-08-27Matthew Boyd 1 ...,False,0 100 1 54 2 86 3 52 4 ...,0 56 1 100 2 75 3 76 4 ...,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi..."


In [32]:
for i in range(0,len(data_wrangled.index)):
    # Check the winteam player counts
    winteam_players = data_wrangled.loc[i,'Winteam Players']
    data_wrangled.loc[i,'Winteam Player Count'] = int(len(winteam_players))
    
    # Check the lossteam player counts
    lossteam_players = data_wrangled.loc[i,'Lossteam Players']
    data_wrangled.loc[i,'Lossteam Player Count'] = int(len(lossteam_players))
    
data_wrangled.head()

,Game Code,Winteam Players,Lossteam Players,GameDraw,Winteam Players %time,Lossteam Players %time,Packed Data,Winteam Player Count,Lossteam Player Count
0,2006-04-03CollingwoodAdelaide,0 1976-12-07Nathan Bassett 1 1977-...,0 1972-07-26Nathan Buckley 1 ...,False,0 100 1 61 2 78 3 97 4 ...,0 81 1 59 2 71 3 68 4 ...,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi...",22.0,22.0
1,2006-04-09AdelaideWest Coast,0 1978-03-24Michael Braun 1 1978...,0 1976-12-07Nathan Bassett 1 1977-...,False,0 91 1 92 2 79 3 98 4 ...,0 100 1 69 2 96 3 90 4 ...,"[[1978-03-24Michael Braun, 1978-06-30Ben Cousi...",22.0,22.0
2,2006-04-16MelbourneAdelaide,0 1976-12-07Nathan Bassett 1 1977-...,0 1988-02-16Clint Bartram 1 1...,False,0 100 1 49 2 90 3 90 4 ...,0 81 1 90 2 100 3 100 4 ...,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi...",22.0,22.0
3,2006-04-22FremantleAdelaide,0 1976-12-07Nathan Bassett 1 1977-...,0 1976-03-01Peter Bell 1 19...,False,0 100 1 66 2 94 3 95 4 ...,0 100 1 69 2 88 3 70 4 ...,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi...",22.0,22.0
4,2006-04-29AdelaideWestern Bulldogs,0 1976-12-07Nathan Bassett 1 1977-...,0 1982-08-27Matthew Boyd 1 ...,False,0 100 1 54 2 86 3 52 4 ...,0 56 1 100 2 75 3 76 4 ...,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi...",22.0,22.0


In [33]:
# So we have some games where the winteam player count is zero
# Should probably drop those games
# More accurately, there should always be 22 players in either winteam or lossteam
data_wrangled = data_wrangled[data_wrangled['Winteam Player Count'] == 22.0]
data_wrangled = data_wrangled[data_wrangled['Lossteam Player Count'] == 22.0]
print(data_wrangled['Winteam Player Count'].unique())
print(data_wrangled['Lossteam Player Count'].unique())

[ 22.]
[ 22.]


In [34]:
# Reindexing data_wrangled because you need to do that after you drop values
data_wrangled.reset_index()
data_wrangled.head()

,Game Code,Winteam Players,Lossteam Players,GameDraw,Winteam Players %time,Lossteam Players %time,Packed Data,Winteam Player Count,Lossteam Player Count
0,2006-04-03CollingwoodAdelaide,0 1976-12-07Nathan Bassett 1 1977-...,0 1972-07-26Nathan Buckley 1 ...,False,0 100 1 61 2 78 3 97 4 ...,0 81 1 59 2 71 3 68 4 ...,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi...",22.0,22.0
1,2006-04-09AdelaideWest Coast,0 1978-03-24Michael Braun 1 1978...,0 1976-12-07Nathan Bassett 1 1977-...,False,0 91 1 92 2 79 3 98 4 ...,0 100 1 69 2 96 3 90 4 ...,"[[1978-03-24Michael Braun, 1978-06-30Ben Cousi...",22.0,22.0
2,2006-04-16MelbourneAdelaide,0 1976-12-07Nathan Bassett 1 1977-...,0 1988-02-16Clint Bartram 1 1...,False,0 100 1 49 2 90 3 90 4 ...,0 81 1 90 2 100 3 100 4 ...,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi...",22.0,22.0
3,2006-04-22FremantleAdelaide,0 1976-12-07Nathan Bassett 1 1977-...,0 1976-03-01Peter Bell 1 19...,False,0 100 1 66 2 94 3 95 4 ...,0 100 1 69 2 88 3 70 4 ...,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi...",22.0,22.0
4,2006-04-29AdelaideWestern Bulldogs,0 1976-12-07Nathan Bassett 1 1977-...,0 1982-08-27Matthew Boyd 1 ...,False,0 100 1 54 2 86 3 52 4 ...,0 56 1 100 2 75 3 76 4 ...,"[[1976-12-07Nathan Bassett, 1977-09-04Rhett Bi...",22.0,22.0


In [35]:
# Do this for every game
iterations = len(data_wrangled.index)

for i in range(0,iterations):
    # create some empty dicts+lists, or if they exist clear them
    winplayers = {}
    lossplayers = {}
    winplaypercent = []
    lossplaypercent = []
    
    for j in range(0,22):
        try:
        # Extract all the winning/losing players and their percent playtime
            winplayers[data_wrangled.loc[i,'Winteam Players'][j]]=playerskill[data_wrangled.loc[i,'Winteam Players'][j]]
            lossplayers[data_wrangled.loc[i,'Lossteam Players'][j]]=playerskill[data_wrangled.loc[i,'Lossteam Players'][j]]
            winplaypercent.append(data_wrangled.loc[i,'Winteam Players %time'][j])
            lossplaypercent.append(data_wrangled.loc[i,'Lossteam Players %time'][j])
        except KeyError:
            # KeyErrors are really annoying and I couldn't fix one here that kept popping up so I just used the Except clause
            # This is probably bad but I think it only happened a couple of times in all of the games so I'm not too worried
            None
    try:
        IsDraw = data_wrangled.loc[i,'GameDraw']
        if IsDraw:
            teamranks = ([1 for x in range(0,22)],[1 for x in range(0,22)])
        else:
            teamranks = ([0 for x in range(0,22)],[1 for x in range(0,22)])
        winplaypercent[:]=[x/100.0 for x in winplaypercent]
        lossplaypercent[:]=[x/100.0 for x in lossplaypercent]
        rated_teams = [winplayers,lossplayers]
        winplayers,lossplayers = ts.rate(rated_teams,weights = [winplaypercent,lossplaypercent], ranks = teamranks)
        playerskill.update(winplayers)
        playerskill.update(lossplayers)
    except KeyError:
        # This was included for the same reason as above: to ignore the KeyErrors that kept popping up
        # Also the fact that it was pretty late and I just wanted it to work
        None

In [36]:
# Break up the dictionary in to a dataframe for analysis
playerskill_df = DataFrame(playerskill.items(),columns=['Player','Rating'])
for i in range(0,len(playerskill_df.index)):
    playerskill_df.loc[i,'mu']=playerskill_df['Rating'][i].mu
    playerskill_df.loc[i,'sigma']=playerskill_df['Rating'][i].sigma
    playerskill_df.loc[i,'skill99CI'] = playerskill_df['Rating'][i].mu - 3*playerskill_df['Rating'][i].sigma
    playerskill_df.loc[i,'Player Name'] = playerskill_df.loc[i,'Player'][10:]
    playerskill_df.loc[i,'Player DOB'] = playerskill_df.loc[i,'Player'][:10]

In [37]:
# Write the playerskill dataframe to csv for further analysis if required
playerskill_df.to_csv('player skill 2006-2016.csv')